<a href="https://colab.research.google.com/github/nyan-tech-learn-more/ML-one-to-two-DataScience/blob/main/Task3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task #03 作業 01/02【實作題】靜態網頁爬蟲實作

In [ ]:
##練習：台灣銀行牌告利率提供了每日的外幣匯率價格，包含美金、港幣、英鎊等；請試著撰寫爬蟲程式將台灣銀行牌告利率的資訊抓下來，並且整理成 Pandas 中的 DataFrame 格式。
# Sample Input：https://rate.bot.com.tw/xrt?Lang=zh-TW（資料來源：台灣銀行牌告利率 ）

# Sample Output：將外幣對台幣匯率整理成 DataFrame

In [40]:
import requests
from bs4 import BeautifulSoup
import pandas

url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'
dfs = pandas.read_html(url) #存成dataframe格式
len(dfs)
dfs[0]#所有的資料

#幣別 現金匯率本行買入 現金匯率本行賣出 即期匯率本行買入 即期匯率本行賣出

currency = dfs[0] #所有的資料
type(currency) #pandas.core.frame.DataFrame 類型
currency = currency.iloc[:,0:5] #全部的row以及0~5的index(col)
currency.columns=["幣別","現金匯率-本行買入","現金匯率-本行賣出","即期匯率-本行買入","即期匯率-本行賣出"] ##自訂欄位名稱
currency['幣別']=currency['幣別'].str.extract('\((\w+)\)')
print(currency)

#加入時間  https://docs.python.org/zh-tw/3/library/datetime.html https://ithelp.ithome.com.tw/articles/10250334
from datetime import datetime
currency['匯入時間'] = datetime.now().strftime("%Y-%m-%d")
print(currency)


#如果你想要將 Pandas DataFrame 的資料寫入 SQLite 資料庫，你可以使用以下程式碼：
import sqlite3
# 建立 SQLite 資料庫連線
conn = sqlite3.connect('currency.db')

# 將 DataFrame 的資料寫入資料庫
currency.to_sql('currency_info', conn, if_exists='replace', index=False)

# 關閉資料庫連線
conn.close()


     幣別 現金匯率-本行買入 現金匯率-本行賣出 即期匯率-本行買入 即期匯率-本行賣出
0   USD     32.14     32.81    32.465    32.615
1   HKD     4.004     4.208     4.125     4.195
2   GBP     39.45     41.57    40.345    40.975
3   AUD     20.74     21.52    20.955      21.3
4   CAD     23.17     24.08      23.5     23.83
5   SGD     23.29      24.2     23.76     23.98
6   CHF     34.59     35.79      35.2     35.59
7   JPY    0.1972      0.21     0.204     0.209
8   ZAR         -         -     1.703     1.793
9   SEK      2.58       3.1      2.91      3.03
10  NZD     18.73     19.58     19.06     19.36
11  THB      0.75      0.94    0.8594    0.9054
12  PHP    0.4975    0.6295         -         -
13  IDR   0.00168   0.00238         -         -
14  EUR     33.94     35.28    34.455    35.055
15  KRW   0.02176   0.02566         -         -
16  VND   0.00104   0.00145         -         -
17  MYR     5.778     7.298         -         -
18  CNY     4.397     4.559     4.464     4.524
     幣別 現金匯率-本行買入 現金匯率-本行賣出 即期匯率-本行買入 即期

In [41]:
import pandas as pd
import sqlite3
#讀取資料庫
conn = sqlite3.connect('currency.db')

#讀取資料表
df = pd.read_sql_query("SELECT * from currency_info", conn)

#關閉連線
conn.close()

# Show DataFrame
print(df)

     幣別 現金匯率-本行買入 現金匯率-本行賣出 即期匯率-本行買入 即期匯率-本行賣出        匯入時間
0   USD     32.14     32.81    32.465    32.615  2024-05-01
1   HKD     4.004     4.208     4.125     4.195  2024-05-01
2   GBP     39.45     41.57    40.345    40.975  2024-05-01
3   AUD     20.74     21.52    20.955      21.3  2024-05-01
4   CAD     23.17     24.08      23.5     23.83  2024-05-01
5   SGD     23.29      24.2     23.76     23.98  2024-05-01
6   CHF     34.59     35.79      35.2     35.59  2024-05-01
7   JPY    0.1972      0.21     0.204     0.209  2024-05-01
8   ZAR         -         -     1.703     1.793  2024-05-01
9   SEK      2.58       3.1      2.91      3.03  2024-05-01
10  NZD     18.73     19.58     19.06     19.36  2024-05-01
11  THB      0.75      0.94    0.8594    0.9054  2024-05-01
12  PHP    0.4975    0.6295         -         -  2024-05-01
13  IDR   0.00168   0.00238         -         -  2024-05-01
14  EUR     33.94     35.28    34.455    35.055  2024-05-01
15  KRW   0.02176   0.02566         -   

## Task #03 作業 01 其他不錯的參考方式

In [ ]:
#example 1

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 發送請求獲取網頁內容
res = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
data = res.text

# 使用 BeautifulSoup 解析 HTML
html = BeautifulSoup(data)
rows = html.find(attrs={"title":"牌告匯率"}).find('tbody').find_all('tr')

# 設定欄位名稱對應
format_title_dict = {
    '幣別': '幣別',
    '本行現金買入': '現金買入',
    '本行現金賣出': '現金賣出',
    '本行即期買入': '即期買入',
    '本行即期賣出': '即期賣出',
}

# 初始化資料列表
titles = ['幣別','本行現金買入','本行現金賣出','本行即期買入','本行即期賣出']
raw_data = [[format_title_dict[t] for t in titles]]

# 迭代每一列,提取所需資料
for tr in rows:
  item = []
  for x in titles:
    if x == '幣別':
      t = tr.find(attrs={"class":"xrt-cur-indent"}).text.strip()
    else:
      t = tr.find(attrs={"data-table":x}).text.strip()
    item.append(t)
  raw_data.append(item)

# 建立 DataFrame
df = pd.DataFrame(raw_data)
df


,0,1,2,3,4
0,幣別,現金買入,現金賣出,即期買入,即期賣出
1,美金 (USD),32.14,32.81,32.465,32.615
2,港幣 (HKD),4.004,4.208,4.125,4.195
3,英鎊 (GBP),39.42,41.54,40.315,40.945
4,澳幣 (AUD),20.73,21.51,20.945,21.29
5,加拿大幣 (CAD),23.15,24.06,23.48,23.81
6,新加坡幣 (SGD),23.28,24.19,23.75,23.97
7,瑞士法郎 (CHF),34.56,35.76,35.17,35.56
8,日圓 (JPY),0.1974,0.2102,0.2042,0.2092
9,南非幣 (ZAR),-,-,1.704,1.794


In [ ]:
import pandas as pd
import requests

# 發送請求獲取網頁內容
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'

# 使用 pandas 直接從 HTML 提取表格數據
table = pd.read_html(url)[0]

# 重組欄位名稱和資料
t1 = table['幣別'].rename({'Unnamed: 1_level_1':'現金匯率本行買入'},axis='columns')
t2 = table['現金匯率'].rename({'本行買入':'現金匯率本行賣出','本行賣出':'即期匯率本行買入'},axis='columns')
t2 = t2.iloc[:,[0,1]]
t3 = table['Unnamed: 4_level_0'].rename({'本行買入':'即期匯率本行賣出'},axis='columns')
table = pd.concat([t1,t2,t3], axis=1)

print(table)

                        幣別 現金匯率本行買入 現金匯率本行賣出 即期匯率本行買入 即期匯率本行賣出
0       美金 (USD)  美金 (USD)    32.14    32.81   32.465   32.615
1       港幣 (HKD)  港幣 (HKD)    4.004    4.208    4.125    4.195
2       英鎊 (GBP)  英鎊 (GBP)    39.42    41.54   40.315   40.945
3       澳幣 (AUD)  澳幣 (AUD)    20.73    21.51   20.945    21.29
4   加拿大幣 (CAD)  加拿大幣 (CAD)    23.15    24.06    23.48    23.81
5   新加坡幣 (SGD)  新加坡幣 (SGD)    23.28    24.19    23.75    23.97
6   瑞士法郎 (CHF)  瑞士法郎 (CHF)    34.56    35.76    35.17    35.56
7       日圓 (JPY)  日圓 (JPY)   0.1972     0.21    0.204    0.209
8     南非幣 (ZAR)  南非幣 (ZAR)        -        -    1.704    1.794
9     瑞典幣 (SEK)  瑞典幣 (SEK)     2.57     3.09      2.9     3.02
10      紐元 (NZD)  紐元 (NZD)    18.72    19.57    19.05    19.35
11      泰幣 (THB)  泰幣 (THB)   0.7485   0.9385   0.8579   0.9039
12  菲國比索 (PHP)  菲國比索 (PHP)   0.4972   0.6292        -        -
13    印尼幣 (IDR)  印尼幣 (IDR)  0.00168  0.00238        -        -
14      歐元 (EUR)  歐元 (EUR)    33.94    35.28   34.455  

In [ ]:
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'


r = requests.get(url)
soup = BeautifulSoup(r.text, 'html5lib')


currency = soup.find_all('div', class_='visible-phone print_hide')
cash_buy_rate = soup.find_all('td', class_='rate-content-cash text-right print_hide', attrs={'data-table': '本行現金買入'})
cash_sell_rate = soup.find_all('td', class_='rate-content-cash text-right print_hide', attrs={'data-table': '本行現金賣出'})
sight_buy_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期買入'})
sight_sell_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期賣出'})

lst = []

# .strip() 移除文本內容中的空白字符（空格、換行符等）
for i in range(len(currency)):
    currency_name = currency[i].text.strip()
    cash_buy = cash_buy_rate[i].text.strip()
    cash_sell = cash_sell_rate[i].text.strip()
    sight_buy = sight_buy_rate[i].text.strip()
    sight_sell = sight_sell_rate[i].text.strip()

    lst.append([currency_name, cash_buy, cash_sell, sight_buy, sight_sell])

df = pd.DataFrame(lst, columns=['幣別', '現金匯率本行買入', '現金匯率本行賣出', '即期匯率本行買入', '即期匯率本行賣出'])

df.head()

,幣別,現金匯率本行買入,現金匯率本行賣出,即期匯率本行買入,即期匯率本行賣出
0,美金 (USD),32.14,32.81,32.465,32.615
1,港幣 (HKD),4.004,4.208,4.125,4.195
2,英鎊 (GBP),39.42,41.54,40.315,40.945
3,澳幣 (AUD),20.73,21.51,20.945,21.29
4,加拿大幣 (CAD),23.15,24.06,23.48,23.81


In [ ]:
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'


r = requests.get(url)
soup = BeautifulSoup(r.text, 'html5lib')


currency = soup.find_all('div', class_='visible-phone print_hide')
cash_buy_rate = soup.find_all('td', class_='rate-content-cash text-right print_hide', attrs={'data-table': '本行現金買入'})
cash_sell_rate = soup.find_all('td', class_='rate-content-cash text-right print_hide', attrs={'data-table': '本行現金賣出'})
sight_buy_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期買入'})
sight_sell_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期賣出'})

lst = []

# .strip() 移除文本內容中的空白字符（空格、換行符等）
for i in range(len(currency)):
    currency_name = currency[i].text.strip()
    cash_buy = cash_buy_rate[i].text.strip()
    cash_sell = cash_sell_rate[i].text.strip()
    sight_buy = sight_buy_rate[i].text.strip()
    sight_sell = sight_sell_rate[i].text.strip()

    lst.append([currency_name, cash_buy, cash_sell, sight_buy, sight_sell])

df = pd.DataFrame(lst, columns=['幣別', '現金匯率本行買入', '現金匯率本行賣出', '即期匯率本行買入', '即期匯率本行賣出'])

print(df.head())

           幣別 現金匯率本行買入 現金匯率本行賣出 即期匯率本行買入 即期匯率本行賣出
0    美金 (USD)    32.14    32.81   32.465   32.615
1    港幣 (HKD)    4.004    4.208    4.125    4.195
2    英鎊 (GBP)    39.42    41.54   40.315   40.945
3    澳幣 (AUD)    20.73    21.51   20.945    21.29
4  加拿大幣 (CAD)    23.15    24.06    23.48    23.81


## Task #03 作業 02